In [1]:
import sys
sys.path.append("../../")
import fibi.src.pie_chart as pie_chart, fibi.src.statistical_test as statistical_test
import pathlib as p
import numpy as np
import tqdm
import pandas as pd
import pyarrow.parquet as pq
root = p.Path("../data/algorithms_out/tsp/data_TSP_Uniformly distr. towns_quad.parquet").resolve()
root_folder = p.Path("../").resolve()
table = pq.read_table(root)
df = table.to_pandas()
df.head()

,num_iter,num_iter_glob,init_cost,final_cost,duration,SEED_GLOB,SEED_PROBLEM,SEED_ASSIGN,DATASET,FI_BI,IMPR,NUM_TOWNS,NUM_DIM,filename,KEY,PROBLEM,init_meth,ratio
0,13.0,33.0,878.90566,374.06583,0.000158,0.0,0.0,0.0,Uniformly distr. towns,BI,RAND,20,2.0,dataset,0.0,TSP,random,0.42560
1,25.0,121.0,1532.47995,432.79368,0.000737,1.0,0.0,0.0,Uniformly distr. towns,BI,RAND,30,2.0,dataset,1.0,TSP,random,0.28241
2,38.0,129.0,1967.04181,520.96771,0.001841,2.0,0.0,0.0,Uniformly distr. towns,BI,RAND,40,2.0,dataset,2.0,TSP,random,0.26485
3,71.0,70.0,1532.47995,486.01255,0.000763,27.0,0.0,0.0,Uniformly distr. towns,FI,RAND,30,2.0,dataset,27.0,TSP,random,0.31714
4,29.0,28.0,878.90566,377.87201,0.000077,26.0,0.0,0.0,Uniformly distr. towns,FI,RAND,20,2.0,dataset,26.0,TSP,random,0.42993


In [2]:
list_cases_per_init = {}
allowed_init = {"RAND", "GREEDY", "GREEDY randomized"}
for initialization, df_init in df.groupby("IMPR"):
    if initialization not in allowed_init:
        continue
    L_cases = []
    Ldiff = []
    for instance, df_instance in df_init.groupby(["NUM_TOWNS"]):
        df_instance.sort_values(by="SEED_GLOB", inplace=True)
        bi = df_instance.query("FI_BI == 'BI'")["final_cost"].to_numpy()
        fi = df_instance.query("FI_BI == 'FI'")["final_cost"].to_numpy()
        init = df_instance.query("FI_BI == 'FI'")["init_cost"].to_numpy()
        diff = (bi-fi)/init
        case = pie_chart.get_case_from_diff(
            diff=diff,
            maximization=False,
            init_random=initialization == "RAND"
        )
        Ldiff.append(np.mean(diff))
        L_cases.append(case)# bi-fi < 0 => bi < fi => bi better
    print(initialization, np.mean(diff))

    assert len(L_cases) > 0, "More than 0 samples needed"
    # We build the input for the pie chart function (one list asssociated to the initialization random)
    list_cases_per_init[initialization] = L_cases


problem = "tsp"
dataset = "randomUniform"
# We create the piechart
figure_text = pie_chart.make_latex_piecharts_figure_for_datasets_of_problem(
    list_cases_per_init=list_cases_per_init,
    problem=problem,
    dataset=dataset,
    template_folder=root_folder / "data" / "templates" / "latex"
)
# And save it
with open(root_folder / "data" / "out_diagrams" / f"{problem}_{dataset}.tex", "w", encoding="utf-8") as fp:
    fp.write(figure_text)

GREEDY -0.011885066746867658
GREEDY randomized 0.0006056729997649197
RAND 0.001170716309909587


: 